<a href="https://colab.research.google.com/github/ferit-osirv/lab6/blob/main/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 6 -  Feature and Object Detection

These laboratory excersises are solved on Google Colab and are save on GitHub repo that is connected to GitHub Classroom.

## Tools You need to use to Submit Assignments

In this document, you will solve tasks. This is a Jupyter Notebook which has the **.ipynb** extension, is an interactive web environment for data analysis, visualization, solution presentations, education, and more.

**Google Colab** is a tool that allows you to run and share Jupyter Notebook files on Google's servers, including the use of Google's CPU, GPU, and TPU resources. Colab is like Google Docs for Jupyter Notebooks. **Google Colab does not automatically save your assignment to GitHub.**

**You use GitHub to save and submit your assignments.** When you accept the assignment through GitHub Classroom, a repository is automatically created on your GitHub account with a copy of the task. This is where you will save your solutions. Saving your solutions submits the tasks for that lab.

## How to Solve the Tasks?
1. Accept the task via the Google Classroom link that you will receive. Google Classroom will create a repository on your account.
2. Go to the newly created repository on your account and click on the .ipynb file, then click Open in Colab.
3. You will solve the tasks in Google Colab.

## How to Save (Submit) Tasks?

1. In Google Colab, click on the Open settings gear icon in the top-right corner.
2. Click on the GitHub tab and check the box for Access private repositories and organizations.
3. A new window will open for you to grant access to GitHub. For ferit-osirv, click Grant.
4. Save and exit the settings.
5. Click on File > Save a copy in GitHub.
6. Select the lab repository that includes your name.

> *Note:* You only need to complete steps 1-4 the first time.

7. Click on **File > Save a copy in GitHub**.
8. Select created repository **koji uključuje vaše ime**.

## Copying Files from the GitHub Repository

For completing the exercises, you will need images and other files that will be stored in the GitHub repository of the exercise. A command like this will be available in the notebook for each exercise. It will copy the files from GitHub to the Google Colab environment.

**You need to run this command before starting each exercise.**

In [1]:
!rm -rf clone && git clone https://github.com/ferit-osirv/lab6 clone && cp -a clone/. .


Cloning into 'clone'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 25 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


**Google Colab will occasionally delete all files**. Therefore, you might need to rerun this command between sessions. If you encounter errors indicating that files do not exist, try running the command again.

# Object detection algorithms (P1)

Broadly speaking, object detection can be broken down into 'traditional' image processing methods, and deep learning-based approaches.

In more traditional based approaches, computer vision techniques are used to look at various features of an image, such as the color histogram or edges, to identify groups of pixels that may belong to an object. 

On the other hand, deep learning-based approaches employ convolutional neural networks (CNNs) to perform end-to-end, unsupervised object detection, in which features don’t need to be defined and extracted separately. 

For now, in this lab we will focus on traditional image detection methods, while deep learning methods which have become the state-of-the-art approaches for object detection, will be covered in further laboratory excersises. 

## Template Matching Algorithm

Template Matching is a method for searching and finding the location of a template image in a larger image. 

In OpenCV, template matching is implemented inside ```cv2.matchTemplate``` function. 

So what exactly is the ```cv2.matchTemplate``` function doing? Essentially, this function takes a “sliding window” of our query image and slides it across our puzzle image from left to right and top to bottom, one pixel at a time. Then, for each of these locations, we compute the correlation coefficient to determine how “good” or “bad” the match is.

Regions with sufficiently high correlation can be considered “matches” for our image template. There are a variety of methods to perform template matching, 
based on different correlation coefficients. 

Definitions of different correlation coefficients can be found at: 
https://docs.opencv.org/2.4/modules/imgproc/doc/object_detection.html




### Assigment 2 - Object detection using template matching

In [24]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 8] # enlarge inline plots
from google.colab.patches import cv2_imshow

In [ ]:
# Load full input image 
full = cv2.imread('images/cat.jpg')
full = cv2.cvtColor(full, cv2.COLOR_BGR2RGB)
plt.imshow(full)

In [ ]:
# Load Template image
# This is a subset of the previous image, note how its actually the exact image
template = cv2.imread('images/cat_face.jpg')
template = cv2.cvtColor(template, cv2.COLOR_BGR2RGB)
plt.imshow(template)

In [27]:
height, width,channels = template.shape

In [28]:
# Define 6 different template matching methods
# TODO:
methods = []

In [ ]:
for m in methods:
    
    # Create a copy of the image
    full_copy = full.copy()
    
    # Get the actual function instead of the string
    method = eval(m)

    # apply template matching algorithm
    res = cv2.matchTemplate(full_copy,template,method)
    
    # grab the Max and Min values, plus their locations
    #TODO:
    min_val, max_val, min_loc, max_loc = 
        
    # if the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
    #TODO: 
    else:
    #TODO: 
        
    # assign the bottom right of the rectangle
    bottom_right = (top_left[0] + width, top_left[1] + height)
    # draw the red rectangle
    cv2.rectangle(full_copy,top_left, bottom_right, 255, 10)

    # Plot the Images
    plt.subplot(1,2,1)
    plt.imshow(res)
    plt.title('Result of Template Matching')
    
    plt.subplot(1,2,2)
    plt.imshow(full_copy)
    plt.title('Detected Point')
    plt.suptitle(m)  
    plt.show()